<a href="https://colab.research.google.com/github/gtmray/Keras/blob/master/Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import categorical_crossentropy

In [34]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num of GPUs: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num of GPUs:  1


In [35]:
pd.set_option('display.max_columns', 10)
sns.set()
plt.style.use('ggplot')

cols = ["Pregnancies" ,"Glucose" ,"BloodPressure" ,"SkinThickness" ,"Insulin" ,"BMI" ,"DiabetesPedigreeFunction" ,"Age" ,"Outcome"]
url = "https://gist.githubusercontent.com/ktisha/c21e73a1bd1700294ef790c56c8aec1f/raw/819b69b5736821ccee93d05b51de0510bea00294/pima-indians-diabetes.csv"
df = pd.read_csv(url, skiprows=9, names=cols)

target_col = 'Outcome'
X = df.drop(target_col, axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = df[target_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential([
                    Dense(units=12, input_dim = 8, activation = 'relu'),
                    Dense(units=16, input_shape=(1, ), activation='relu'),
                    Dense(units=32, activation='relu'),
                    Dense(units=2, activation='softmax')
])
model.summary()
model.compile(optimizer=SGD(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x=X_train, y=y_train, validation_split=0.1, batch_size=5, epochs=100, shuffle=True, verbose=2)
prediction_proba = model.predict(x=X_test, batch_size=5, verbose=0)
prediction = np.argmax(prediction_proba, axis=-1)
print((y_test==prediction).mean())

SAVING AND LOADING
 THE MODEL

In [41]:
####USING .SAVE NORMAL

import os.path
if os.path.isfile('/diabetes_model.h5') is False:
  model.save('/diabetes_model.h5') #Saving here saves architecture, weights, loss, optimizer, state of optimizer

from tensorflow.keras.models import load_model
new_model1 = load_model('/diabetes_model.h5')

new_model1.summary()
new_model1.get_weights()
new_model1.optimizer

In [ ]:
#SAVING IN JSON and yaml

json_string = model.to_json() #Only saves model architecture
yaml_string = model.to_yaml()

from tensorflow.keras.models import model_from_json, model_from_yaml
new_model2 = model_from_json(json_string)
#new_model2.summary()

new_model3 = model_from_yaml(yaml_string)
#new_model3.summary()

In [46]:
#SAVING ONLY WEIGHTS

import os.path
if os.path.isfile('/diabetes_model_weights.h5') is False:
  model.save_weights('/diabetes_model_weights.h5') #Saving here saves architecture, weights, loss, optimizer, state of optimizer

model2 = Sequential([
                    Dense(units=12, input_dim = 8, activation = 'relu'),
                    Dense(units=16, input_shape=(1, ), activation='relu'),
                    Dense(units=32, activation='relu'),
                    Dense(units=2, activation='softmax')
])

model2.load_weights('/diabetes_model_weights.h5') #Shape must match